In [221]:
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt

## Problem 1. Neural Net from Scratch

### Dense Layer (Feed Forward Layer)

In [237]:
class Dense():
    def __init__(self, in_dim, out_dim, bias_flag=True):
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.bias_flag = bias_flag
        self.test = False

        self.weight = np.random.rand(in_dim, out_dim)
        self.bias = np.random.rand(1, out_dim)

        self.weight_grad = np.zeros((in_dim, out_dim))
        self.bias_grad = np.zeros((1, out_dim))

    def __call__(self, input, test=False):
        assert isinstance(input, np.ndarray), 'Input must be an array'
        #assert input.shape[
        #    1] == self.in_dim, 'Dimensions mismatch {0} != {1}'.format(
         #       input.shape[1], self.in_dim)

        self.input = input
        return self.forward(input)

    def init_weight(self):
        self.weight.fill(0.01)

    def init_bias(self):
        self.bias.fill(0.01)
        
    def forward(self, input):
        self.batch_size = input.shape[0]
        ss=[]
        if not self.bias_flag:
            for i in range(self.batch_size):
                
                print(self.weight.shape)
                ss.append(np.dot(self.weight.T,self.input[i]))
            return ss
        else:
            for i in range(self.batch_size):
                #print(input[i].shape)
                #print(self.weight.T.shape)
                #print(self.bias.shape)
                
                ss.append(np.dot(np.atleast_2d(self.input[i]),self.weight)+self.bias)
                # print('a'*5)

            return ss

    def backward(self, backprop_err):
        for i in range(self.batch_size):
            #print(self.input[i].shape,'inputshape')
            #print(self.weight_grad[i].shape,'grad_i')
            #print(backprop_err[i].shape,'backproperr_i')
            #print(backprop_err,'backproperr_itself')
           # print(backprop_err[i],'backproperr_itself_i')
            try:
                self.weight_grad += np.dot(np.atleast_2d(self.input[i]).T,backprop_err[i][0].reshape(-1,1).T)/self.batch_size
            except ValueError as e:
                #print(e)
                self.weight_grad += np.dot(np.atleast_2d(self.input[i]),backprop_err[i][0].reshape(-1,1).T)/self.batch_size
            #print(self.bias_grad)
            self.bias_grad += np.sum(backprop_err[i][0])/self.batch_size
        return backprop_err, self.weight

    def update(self, lr):
        self.weight += lr * self.weight_grad
        self.bias += lr * self.bias_grad

### Activation Functions

In [223]:
class Sigmoid():
    def __init__(self, test=False):
        self.grad = None
        self.test= test

    def __call__(self, input, test=None):
        out = 1 / (1 + np.exp(-1 * input))
        if test is None: test = self.test
        if not test:
            self.grad = out * (1 - out)
        return out
    
    def backward(self, backprop_err, weight=None):
        if weight is None:
            return backprop_err * self.grad
        else:
            return (backprop_err @ weight.T) * self.grad

In [224]:
class Linear():
    def __init__(self, test=False):
        self.grad = None
        self.test= test

    def __call__(self, input, test=None):
        out = input
        if test is None: test = self.test
        if not test:
            self.grad = np.ones_like(out)
        return out
    
    def backward(self, backprop_err, weight=None):
        if weight is None:
            return backprop_err * self.grad
        else:
            return (backprop_err @ weight.T) * self.grad

In [225]:
class Relu():
    def __init__(self, test=False):
        self.grad = None
        self.test= test

    def __call__(self, input, test=None):
        out = np.maximum(input,0)
        if test is None: test = self.test
        if not test:
            self.grad = np.greater(out,0)*1
        return out
    
    def backward(self, backprop_err, weight=None):
        if weight is None:
            return backprop_err * self.grad
        else:
            return (backprop_err @ weight.T) * self.grad

In [226]:
class Tanh():
    def __init__(self, test=False):
        self.grad = None
        self.test= test
    
    def __call__(self, input, test=None):
        out = (np.exp(input)-np.exp(-input))/(np.exp(input)+np.exp(-input))
        if test is None: test = self.test
        if not test:
            self.grad = 1-(np.exp(input)-np.exp(-input))/(np.exp(input)+np.exp(-input))**2
        return out
    
    def backward(self, backprop_err, weight=None):
        if weight is None:
            return backprop_err * self.grad
        else:
            return (backprop_err @ weight.T) * self.grad

### Loss Metrics

In [227]:
class MSE():
    def __init__(self):
        self.grad = None

    def __call__(self, y_pred, y_true):
        error = np.sum(np.sum((y_pred - y_true)**2, axis=0))
        self.grad = 2*(y_pred - y_true)
        return error

    def backward(self):
        return self.grad

### Model

In [228]:
class SequentialModel:
    def __init__(self, *layers):
        self.layers = []
        self.updatable = []

        for layer in layers:
            self.layers.append(layer)
            if hasattr(layer, 'update'):
                if callable(layer.update):
                    layer.init_weight()
                    layer.init_bias()
                    self.updatable.append(layer)

    def __call__(self, input, test=None):
        forwarded = input
        for layer in self.layers:
            forwarded = layer(forwarded, test)
        return forwarded

    def backward(self, criterion):
        backwarded = criterion.backward()

        for layer in reversed(self.layers):
            #print(layer)
            if isinstance(backwarded, tuple):
                backwarded = layer.backward(*backwarded)
            else:
                backwarded = layer.backward(backwarded)
    
    def update(self, lr):
        for layer in self.updatable:
            layer.update(lr)

class Model:
    def __init__(self):
        self.seq = SequentialModel(Dense(2, 4), Relu(),
        Dense(4, 4),
        Relu(),
        Dense(4, 1),
        Relu())

    def __call__(self, input, test=None):
        return self.forward(input, test)

    def forward(self, input, test):      
        self.y_pred = self.seq(input)
        return self.y_pred

    def backward(self):
        self.seq.backward(self.criterion)

    def update(self, lr):
        self.seq.update(lr)
            
    def loss(self, criterion, y_true):
        self.criterion = criterion
        return self.criterion(self.y_pred, y_true)

In [229]:
model = Model()
mean_square_err = MSE()

for epoch in range(7):
    print("Epoch:", epoch, "~"*50)
    input, target = np.array([[1, 2], [4, 1.3], [1, 3],[1.5, 2], [4, 1.3], [1, 3]]), np.array([[3.75], [9],[8]])
    print(np.array([[4], [9],[8]]).shape)
    print(np.array([[1, 2], [4, 1.3], [1, 3]]).shape)
    batch_size = input.shape[0]*2
    out = model(input)
    print("model output:", *out)
    loss = model.loss(mean_square_err, target) / batch_size
    print("*"*5, "Loss:", loss)
    model.backward()
    model.update(lr=0.1)

Epoch: 0 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(3, 1)
(3, 2)
model output: [[0.010464]] [[0.0105008]] [[0.01048]] [[0.010472]] [[0.0105008]] [[0.01048]]
***** Loss: 79.31389397145365
Epoch: 1 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(3, 1)
(3, 2)
model output: [[0.76107802]] [[0.76115529]] [[0.76111164]] [[0.76109482]] [[0.76115529]] [[0.76111164]]
***** Loss: 64.60700645570695
Epoch: 2 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(3, 1)
(3, 2)
model output: [[2.12867001]] [[2.1289436]] [[2.12878919]] [[2.12872951]] [[2.1289436]] [[2.12878919]]
***** Loss: 42.15617835180655
Epoch: 3 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(3, 1)
(3, 2)
model output: [[3.90625209]] [[3.90722429]] [[3.90667635]] [[3.90646362]] [[3.90722429]] [[3.90667635]]
***** Loss: 21.36020544455073
Epoch: 4 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(3, 1)
(3, 2)
model output: [[5.71166357]] [[5.71363027]] [[5.71252276]] [[5.71209159]] [[5.71363027]] [[5.71252276]]
****

### *Testing Our Model Against Pytorch Model*

In [230]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

    def init_weights(self):
        self.fc1.weight.data.fill_(0.01)
        self.fc1.bias.data.fill_(0.01)
        self.fc2.weight.data.fill_(0.01)
        self.fc2.bias.data.fill_(0.01)
        self.fc3.weight.data.fill_(0.01)
        self.fc3.bias.data.fill_(0.01)


net = Net()

In [231]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0, dampening=0, weight_decay=0, nesterov=False)

In [232]:
net.init_weights()

In [233]:
for epoch in range(7):
    print("Epoch:", epoch, "~"*50)
    inputs, labels = torch.tensor([[1.0, 2.0], [4.0, 1.3]]), torch.tensor([[4.0], [9.0]])
    optimizer.zero_grad()
    outputs = net(inputs)
    print("model output:", *outputs.data)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print("*"*5, "Loss:", loss.item())

Epoch: 0 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model output: tensor([0.0105]) tensor([0.0105])
***** Loss: 48.363746643066406
Epoch: 1 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model output: tensor([1.3105]) tensor([1.3106])
***** Loss: 33.18022155761719
Epoch: 2 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model output: tensor([2.3568]) tensor([2.3572])
***** Loss: 23.413333892822266
Epoch: 3 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model output: tensor([3.2124]) tensor([3.2138])
***** Loss: 17.049976348876953
Epoch: 4 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model output: tensor([3.9408]) tensor([3.9468])
***** Loss: 12.769041061401367
Epoch: 5 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model output: tensor([4.6128]) tensor([4.6426])
***** Loss: 9.681167602539062
Epoch: 6 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model output: tensor([5.3309]) tensor([5.4963])
***** Loss: 7.023585319519043


## Problem 3. Iris Dataset Classification using Neural Net

In [234]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = torch.tensor(np.array(load_iris()['data']))
target= torch.tensor(load_iris()['target']).reshape(-1,1)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

    def init_weights(self):
        self.fc1.weight.data.fill_(0.01)
        self.fc1.bias.data.fill_(0.01)
        self.fc2.weight.data.fill_(0.01)
        self.fc2.bias.data.fill_(0.01)
        self.fc3.weight.data.fill_(0.01)
        self.fc3.bias.data.fill_(0.01)

net = Net()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0, dampening=0, weight_decay=0, nesterov=False)
net.init_weights()
print(data.shape)
print(target.shape)
for epoch in range(7):
    print("Epoch:", epoch, "~"*50)
    inputs, labels = data, target
    optimizer.zero_grad()
    outputs = net(inputs.float())
    print("model output:", *outputs.data)
    loss = criterion(outputs.float(), labels.float())
    loss.backward()
    optimizer.step()
    print("*"*5, "Loss:", loss.item())

torch.Size([150, 4])
torch.Size([150, 1])
Epoch: 0 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model output: tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0106]) tensor([0.0107]) tensor([0.0107]) 

## Problem 4. Iris Dataset Classification using Neural Net with PCA

In [238]:
from sklearn.decomposition import PCA
data = load_iris()['data']
target= load_iris()['target'].reshape(-1,1)
sklearn_pca = PCA(n_components=2)
data_new = sklearn_pca.fit_transform(data)
print(data_new.shape)
print(target.shape)
model = Model()
mean_square_err = MSE()

for epoch in range(10):
    print("Epoch:", epoch, "~"*50)
    input, target = data_new, target
    batch_size = input.shape[0]
    out = model(input)
    # print("model output:", *out)
    loss = model.loss(mean_square_err, target) / batch_size
    print("*"*5, "Loss:", loss)
    model.backward()
    model.update(lr=0.1)

(150, 2)
(150, 1)
Epoch: 0 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 246.88908373957412
Epoch: 1 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 246.269543003193
Epoch: 2 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 244.91088828129563
Epoch: 3 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 242.55386725986293
Epoch: 4 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 238.75855484482133
Epoch: 5 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 232.84348626179602
Epoch: 6 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 223.81571232563715
Epoch: 7 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 210.33392515952906
Epoch: 8 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 190.83416095585488
Epoch: 9 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
***** Loss: 164.17771374563992
